In [ ]:
import copy
import random
random.seed(42)
import datetime
import heapq
from collections import namedtuple

In [ ]:
class Item:
    def __init__(self, item: str, utility: int):
        self.item = item
        self.utility = utility
        self._twu = 0

    @property
    def twu(self) -> int:
        return self._twu

    @twu.setter
    def twu(self, value: int) -> None:
        self._twu = value

    def __repr__(self):
        return f"{self.item}"

    def __eq__(self, other):
        if isinstance(other, Item):
            return self.item == other.item and self.utility == other.utility
        return False

    def __hash__(self):
        return hash((self.item, self.utility))

def check_order_condition(a: Item, b: Item) -> bool:
    if a.utility <= 0 and b.utility > 0:
        return True
    elif a.utility * b.utility >= 0:
        return a.twu >= b.twu
    return False


def check_order_item_and_set(ik: Item, X: set[Item]) -> bool:
    for i in X:
        if i != ik and check_order_condition(ik, i) == False:
            return False
    return True

class TransItem:
    def __init__(self, item: Item, quantity: int, probability: float):
        self.item = item
        self.quantity = quantity
        self.probability = probability

    def __repr__(self):
        return f"{self.item},{self.quantity},{self.probability}"

    def get_total_probability(self):
        return self.quantity * self.probability

class Transaction:
    def __init__(self, id: int, trans_items: list[TransItem]):
        self.id = id
        self.trans_items = trans_items

    def __repr__(self):
        return f"t{self.id}, {self.trans_items}"

    def contains_item_set(self, item_set: set[Item]) -> bool:
        transaction_items = {trans_item.item for trans_item in self.trans_items}
        return item_set.issubset(transaction_items)

    def get_quantity_of_item(self, item: Item) -> int:
        for trans_item in self.trans_items:
            if trans_item.item == item:
                return trans_item.quantity
        return 0

    def get_probability_of_item(self, item: Item) -> int:
        for trans_item in self.trans_items:
            if trans_item.item == item:
                return trans_item.probability
        return 0

    def get_items(self) -> set[Item]:
        return {trans_item.item for trans_item in self.trans_items}

    def get_probability_of_item_set(self, item_set: set[Item]) -> float:
        total_probability = 0.0
        if self.contains_item_set(item_set):
            total_probability = 1.0
            for trans_item in self.trans_items:
                if trans_item.item in item_set:
                    total_probability *= trans_item.probability
        return total_probability

    def get_positive_utility_of_item_set(self, item_set: set[Item]):
        pu = 0
        if self.contains_item_set(item_set):
            for item in item_set:
                utility = item.utility
                if utility > 0:
                    quantity = self.get_quantity_of_item(item)
                    pu += item.utility * quantity
        return pu

    def get_negative_utility_of_item_set(self, item_set: set[Item]):
        nu = 0
        if self.contains_item_set(item_set):
            for item in item_set:
                utility = item.utility
                if utility < 0:
                    quantity = self.get_quantity_of_item(item)
                    nu += item.utility * quantity
        return nu

    def get_utility_of_item_set(self, item_set: set[Item]):
        u = 0
        if self.contains_item_set(item_set):
            for item in item_set:
                quantity = self.get_quantity_of_item(item)
                u += item.utility * quantity
        return u

    def get_remaining_utility_of_item_set(self, items: set[Item]):
        ru = 0
        trans_item_set = self.get_items()
        for item in trans_item_set:
            if item.utility > 0:
                if item not in items:
                    if check_order_item_and_set(item, items) == True:
                        ru += item.utility * self.get_quantity_of_item(item)
        return ru

class PriorityQueue:
    def __init__(self, max_size: int):
        self.max_size = max_size
        self.heap = []

    def push(self, utility: int, item_set: set[Item]):
        if len(self.heap) < self.max_size:
            heapq.heappush(self.heap, (utility, item_set))
        else:
            if utility > self.heap[0][0]:
                heapq.heappushpop(self.heap, (utility, item_set))

    def sort(self):
        return sorted(self.heap, reverse=True)

    def print_items(self):
        for utility, item_set in sorted(self.heap, reverse=True):
            print(f"{item_set}: {utility}")

Utilities = namedtuple("Utilities", ["tid", "pro", "pu", "nu", "ru"])

class AbstractList:
    def __init__(self, items: set[Item], utility_values: list[Utilities]):
        self.items = items
        self.utility_values = utility_values

    def get_ru(self):
        ru = 0
        for i in self.utility_values:
            ru += i.ru
        return ru

    def get_pu(self):
        pu = 0
        for i in self.utility_values:
            pu += i.pu
        return pu

    def get_nu(self):
        nu = 0
        for i in self.utility_values:
            nu += i.nu
        return nu

    def get_pro(self):
        pro = 0
        for i in self.utility_values:
            pro += i.pro
        return pro

    def __repr__(self):
        if not self.utility_values:
            return "Empty PNU-List"

        # Column titles
        titles = ["PRO", "PU", "NU", "RU"]

        # Get the number of columns from the first utility value
        if isinstance(self.utility_values[0], (list, tuple)):
            num_columns = len(self.utility_values[0])
        else:
            num_columns = 1

        # Create combined items string
        items_str = ",".join(str(item) for item in self.items)
        items_str = "(" + items_str + ")"

        value_widths = []
        for i in range(num_columns):
            max_width = max(
                (
                    len(str(round(row[i], 3)))
                    if isinstance(row, (list, tuple))
                    else len(str(row))
                )
                for row in self.utility_values
            )
            if i == 0:
                max_width = max(max_width, len(items_str))
            else:
                max_width = max(max_width, len(titles[i - 1]))
            value_widths.append(max_width)

        result = []

        total_width = sum(value_widths) + 3 * num_columns + 1
        result.append("-" * total_width)

        row = "|"
        row += f" {items_str.rjust(value_widths[0])} |"
        for i in range(1, num_columns):
            row += f" {titles[i-1].center(value_widths[i])} |"
        result.append(row)

        result.append("-" * total_width)

        for utility in self.utility_values:
            row = "|"
            if isinstance(utility, (list, tuple)):
                for i, value in enumerate(utility):
                    row += f" {str(round(value, 3)).rjust(value_widths[i])} |"
            else:
                row += f" {str(utility).rjust(value_widths[0])} |"
            result.append(row)

        result.append("-" * total_width)
        return "\n".join(result)

In [ ]:
# # a = Item("a", 4)
# # b = Item("b", 3)
# # c = Item("c", 1)
# # d = Item("d", -1)
# # e = Item("e", 2)

# # t1_trans_items = [
# #     TransItem(a, 1, 0.0),
# #     TransItem(b, 2, 0.0),
# #     TransItem(c, 2, 0.0),
# #     TransItem(d, 1, 0.0),
# # ]
# # t2_trans_items = [
# #     TransItem(a, 1, 0.0),
# #     TransItem(b, 3, 0.0),
# #     TransItem(c, 3, 0.0),
# #     TransItem(d, 2, 0.0),
# #     TransItem(e, 2, 0.0),
# # ]

# # t3_trans_items = [
# #     TransItem(a, 1, 0.0),
# #     TransItem(c, 6, 0.0),
# #     TransItem(e, 3, 0.0),
# # ]

# # t4_trans_items = [
# #     TransItem(b, 3, 0.0),
# #     TransItem(d, 5, 0.0),
# #     TransItem(e, 2, 0.0),
# # ]

# # t5_trans_items = [
# #     TransItem(b, 1, 0.0),
# #     TransItem(c, 5, 0.0),
# #     TransItem(d, 1, 0.0),
# #     TransItem(e, 4, 0.0),
# # ]

# # t6_trans_items = [
# #     TransItem(c, 2, 0.0),
# #     TransItem(d, 1, 0.0),
# #     TransItem(e, 1, 0.0),
# # ]

# # t1 = Transaction(1, t1_trans_items)
# # t2 = Transaction(2, t2_trans_items)
# # t3 = Transaction(3, t3_trans_items)
# # t4 = Transaction(4, t4_trans_items)
# # t5 = Transaction(5, t5_trans_items)
# # t6 = Transaction(6, t6_trans_items)

# # database1 = [t1, t2, t3, t4, t5, t6]
# # database2 = [t1, t2, t3, t4, t5, t6]

# # item_list = {a, b, c, d, e}


# a = Item("a", 6)
# b = Item("b", 7)
# c = Item("c", 1)
# d = Item("d", -5)
# e = Item("e", 3)

# item_list = [a, b, c, d, e]

# t1_trans_items = {TransItem(b, 3, 0.85), TransItem(c, 1, 1.0), TransItem(d, 2, 0.70)}

# t2_trans_items = {
#     TransItem(a, 1, 1.0),
#     TransItem(b, 1, 0.60),
#     TransItem(c, 3, 0.75),
#     TransItem(e, 1, 0.40),
# }

# t3_trans_items = {
#     TransItem(a, 1, 0.55),
#     TransItem(b, 2, 0.60),
#     TransItem(c, 4, 1.0),
#     TransItem(d, 1, 0.90),
#     TransItem(e, 5, 0.40),
# }

# t4_trans_items = {TransItem(b, 3, 0.90), TransItem(d, 1, 0.45)}

# t5_trans_items = {
#     TransItem(a, 4, 1.0),
#     TransItem(c, 3, 0.85),
#     TransItem(d, 2, 0.70),
#     TransItem(e, 2, 0.45),
# }

# t1 = Transaction(1, t1_trans_items)
# t2 = Transaction(2, t2_trans_items)
# t3 = Transaction(3, t3_trans_items)
# t4 = Transaction(4, t4_trans_items)
# t5 = Transaction(5, t5_trans_items)
# db1 = [t1, t2, t3, t4, t5]
# db2 = [t1, t2, t3, t4, t5]


a = Item("a", 5)
b = Item("b", 2)
c = Item("c", 1)
d = Item("d", 2)
e = Item("e", 3)
f = Item("f", 1)
g = Item("g", 1)

# Step 2: Create Transactions with TransItems
# Transaction T1
t1_trans_items = {
    TransItem(a, 1, 0.0),
    TransItem(c, 1, 0.0),
    TransItem(d, 1, 0.0),
}
t1 = Transaction(1, t1_trans_items)

# Transaction T2
t2_trans_items = {
    TransItem(a, 2, 0.0),
    TransItem(c, 6, 0.0),
    TransItem(e, 2, 0.0),
    TransItem(g, 5, 0.0),
}
t2 = Transaction(2, t2_trans_items)

# Transaction T3
t3_trans_items = {
    TransItem(a, 1, 0.0),
    TransItem(b, 2, 0.0),
    TransItem(c, 1, 0.0),
    TransItem(d, 6, 0.0),
    TransItem(e, 1, 0.0),
    TransItem(f, 5, 0.0),
}
t3 = Transaction(3, t3_trans_items)

# Transaction T4
t4_trans_items = {
    TransItem(b, 4, 0.0),
    TransItem(c, 3, 0.0),
    TransItem(d, 3, 0.0),
    TransItem(e, 1, 0.0),
}
t4 = Transaction(4, t4_trans_items)

# Transaction T5
t5_trans_items = {
    TransItem(b, 2, 0.0),
    TransItem(c, 2, 0.0),
    TransItem(e, 1, 0.0),
    TransItem(g, 2, 0.0),
}
t5 = Transaction(5, t5_trans_items)


database = [t1, t2, t3, t4, t5]

In [ ]:
def find_trans_ids_contain_item_set(item_set: set[Item], db: list[Transaction]):
    transactionsList: list[int] = list()
    for trans in db:
        if item_set.issubset(trans.items_quantities.keys()):
            transactionsList.append(trans.id)
    return transactionsList

def find_max_min_avg_periodic_of_item_set(item_set: set[Item], db: list[Transaction]):
    trans_ids: list[int] = find_trans_ids_contain_item_set(item_set, db)
    print(trans_ids)
    m = len(trans_ids)
    ps: list[int] = list()
    ps.append(trans_ids[0])
    for i in range(1, m):
        ps.append(trans_ids[i] - trans_ids[i - 1])
    ps.append(len(db) - trans_ids[m - 1])
    print(ps)
    max_per, min_per = max(ps), min(ps)
    avg_per = len(db) / len(ps)
    return max_per, min_per, avg_per

In [ ]:
def calculate_positive_utility_of_transaction(trans: Transaction):
    pu = 0
    items: set[Item] = trans.get_items()
    for item in items:
        if item.utility > 0:
            pu += trans.get_quantity_of_item(item) * item.utility
    return pu

def calculate_transaction_weight_utility(items: set[Item], database: list[Transaction]):
    twu = 0
    for trans in database:
        if trans.contains_item_set(items):
            twu += calculate_positive_utility_of_transaction(trans)
    return twu


def calculate_probability_and_twu_of_item_set(
    item_set: set[Item], database: list[Transaction]
) -> tuple[float, int]:
    p = 0
    twu = 0
    for trans in database:
        if trans.contains_item_set(item_set):
            p += trans.get_probability_of_item_set(item_set)
            twu += calculate_positive_utility_of_transaction(trans)
    return p, twu


def calculate_probability_and_utility_of_item_set(
    item_set: set[Item], database: list[Transaction]
) -> tuple[float, int]:
    p = 0
    u = 0
    for trans in database:
        if trans.contains_item_set(item_set):
            p += trans.get_probability_of_item_set(item_set)
            u += trans.get_utility_of_item_set(item_set)
    return p, u

In [ ]:
def sort_items_by_twu_and_utility(items: list[Item]) -> list[Item]:
    def sort_key(item: Item) -> tuple:
        return (0 if item.utility > 0 else 1, item.twu)
    return sorted(items, key=sort_key)

In [ ]:
def calculate_utility_of_item_set_in_database(
    item_set: set[Item], database: list[Transaction]
) -> int:
    """_summary_

    Args:
        item_set (set[Item]): set items
        database (list[Transaction]): database

    Returns:
        utility: the utility of item set in database
    """
    utility = 0
    for trans in database:
        if trans.contains_item_set(item_set):
            utility += trans.get_utility_of_item_set(item_set)
    return utility

In [ ]:
def calculate_utilities_of_item_set(
    item_set: set[Item], database: list[Transaction]
) -> tuple[float, int, int, int]:
    p = 0
    u = 0
    ru = 0
    twu = 0
    for trans in database:
        if trans.contains_item_set(item_set):
            p += trans.get_probability_of_item_set(item_set)
            u += trans.get_positive_utility_of_item_set(
                item_set
            ) + trans.get_negative_utility_of_item_set(item_set)
            ru += trans.get_remaining_utility_of_item_set(item_set)
            twu += calculate_positive_utility_of_transaction(trans)
    return p, u, ru, twu

**Algorithm 1: Positive Real Item Utility strategy**


In [ ]:
def priu_pruning(priu_list: list[Item], k: int, database: list[Transaction]) -> int:
    """_summary_

    Args:
        priu_list (list[Item]): _description_
        k (int): _description_
        database (list[Transaction]): _description_
    Returns:
        int: _description_
    """
    priu_values: list[int] = [0 for i in range(0, len(priu_list))]
    for i in range(0, len(priu_list)):
        priu_value: int = calculate_utility_of_item_set_in_database(
            {priu_list[i]}, database
        )
        priu_values[i] = priu_value
    priu_values.sort(reverse=True)
    min_utility = (
        priu_values[k - 1] if k <= len(priu_list) else priu_values[len(priu_list) - 1]
    )
    return min_utility

**Algorithm 2: Positive LIU-Exact strategy**


In [ ]:
def pliue_strategy(lius: dict[frozenset[Item], int], k: int, current_min_util: int):
    piqu_liu = set()

    for key, utility in lius.items():
        piqu_liu.add(utility)

    top_k = list(piqu_liu)
    top_k.sort(reverse=True)

    max_index = len(top_k) - 1 if k > len(top_k) else k - 1
    if top_k[max_index] > current_min_util:
        current_min_util = top_k[max_index]

    return current_min_util

In [ ]:
def create_liu_dict(
    item_list: list[Item], database: list[Transaction]
) -> dict[frozenset[Item], int]:
    n = len(item_list)
    eucs_dict = {}
    for i in range(n):
        for j in range(i + 1, n):
            item_pair = frozenset({item_list[i], item_list[j]})
            utility = calculate_utility_of_item_set_in_database(item_pair, database)
            eucs_dict[item_pair] = utility
    return eucs_dict

**Algorithm 3: PLIU_LB strategy**


In [ ]:
def pliu_lb_strategy(
    lius: dict[frozenset[Item], int],
    piqu_liu: set[int],
    ordered_list: list[Item],
    k: int,
    current_min_util: int,
    database: list[Transaction]
) -> int:
    piqu_lb_liu: set[int] = set()
    for key, value in lius.items():
        key_list = list(key)
        post_start_item: Item = (
            key_list[0]
            if check_order_condition(key_list[1], key_list[0])
            else key_list[1]
        )

        post_end_item: Item = (
            key_list[1]
            if check_order_condition(key_list[1], key_list[0])
            else key_list[0]
        )

        start_index = ordered_list.index(post_start_item)
        end_index = ordered_list.index(post_end_item)

        sub_list = ordered_list[start_index + 1 : end_index]
        # print(str(post_start_item) + " -> " + str(post_end_item) + ": " + str(sub_list))

        for x in range(start_index + 1, end_index):
            # print("x = " + str(ordered_list[x]))
            util_lb = value - calculate_utility_of_item_set_in_database(
                {ordered_list[x]}, database
            )
            if util_lb > current_min_util:
                piqu_lb_liu.add(util_lb)
            for y in range(x + 1, end_index):
                # print("y = " + str(ordered_list[y]))
                util_lb = (
                    value
                    - calculate_utility_of_item_set_in_database(
                        {ordered_list[x]}, database
                    )
                    - calculate_utility_of_item_set_in_database(
                        {ordered_list[y]}, database
                    )
                )
                if util_lb > current_min_util:
                    piqu_lb_liu.add(util_lb)
                for z in range(y + 1, end_index):
                    # print("z = " + str(ordered_list[z]))
                    util_lb = (
                        value
                        - calculate_utility_of_item_set_in_database(
                            {ordered_list[x]}, database
                        )
                        - calculate_utility_of_item_set_in_database(
                            {ordered_list[y]}, database
                        )
                        - calculate_utility_of_item_set_in_database(
                            {ordered_list[z]}, database
                        )
                    )
                    if util_lb > current_min_util:
                        piqu_lb_liu.add(util_lb)
                    for w in range(z + 1, end_index):
                        # print("w = " + str(ordered_list[w]))
                        util_lb = (
                            value
                            - calculate_utility_of_item_set_in_database(
                                {ordered_list[x]}, database
                            )
                            - calculate_utility_of_item_set_in_database(
                                {ordered_list[y]}, database
                            )
                            - calculate_utility_of_item_set_in_database(
                                {ordered_list[z]}, database
                            )
                            - calculate_utility_of_item_set_in_database(
                                {ordered_list[w]}, database
                            )
                        )
                        if util_lb > current_min_util:
                            piqu_lb_liu.add(util_lb)
    piqu_all: list[int] = list(piqu_lb_liu | piqu_liu)
    if len(piqu_all) >= k:
        piqu_all = list(piqu_all)
        piqu_all.sort(reverse=True)
        current_min_util = piqu_all[0]
    return current_min_util

In [ ]:
def calculate_local_utility(alpha: set[Item], item: Item, database: list[Transaction]):
    lu = 0
    for trans in database:
        if trans.contains_item_set(alpha | {item}):
            lu += trans.get_utility_of_item_set(
                alpha
            ) + trans.get_remaining_utility_of_item_set(alpha)
    return lu


def calculate_subtree_utility(alpha: set[Item], item: Item, db: list[Transaction]):
    su = 0
    if check_order_item_and_set(item, alpha):
        for trans in db:
            if trans.contains_item_set(alpha | {item}):
                su += (
                    trans.get_utility_of_item_set(alpha)
                    + trans.get_utility_of_item_set({item})
                    + trans.get_remaining_utility_of_item_set(set(alpha | {item}))
                )
    return su

In [ ]:
def calculate_utilities_of_item_set(
    item_set: set[Item], database: list[Transaction]
) -> tuple[float, int, int, int]:
    p = 0
    u = 0
    ru = 0
    twu = 0
    for trans in database:
        if trans.contains_item_set(item_set):
            p += trans.get_probability_of_item_set(item_set)
            u += trans.get_positive_utility_of_item_set(
                item_set
            ) + trans.get_negative_utility_of_item_set(item_set)
            ru += trans.get_remaining_utility_of_item_set(item_set)
            twu += calculate_positive_utility_of_transaction(trans)
    return p, u, ru, twu

In [ ]:
def find_database_projection(item_set: set[Item], database: list[Transaction]):
    db_projection = list()
    for trans in database:
        if trans.contains_item_set(item_set):
            db_projection.append(trans)
    return db_projection

In [ ]:
def efim_global_search(
    alpha: set[Item],
    project_db: list[Transaction],
    primary: list[Item],
    secondary: list[Item],
    min_util: int,
    user_prob_threshold: float,
    k: int,
    topk_queue: PriorityQueue,
):
    for pri_item in primary:
        beta: set[Item] = alpha | {pri_item}
        prob, util = calculate_probability_and_utility_of_item_set(beta, project_db)
        if util >= min_util and round(prob, 3) >= user_prob_threshold:
            if frozenset(beta) not in {frozenset(item[1]) for item in topk_queue.heap}:
                topk_queue.push(util, beta)
                if len(topk_queue.heap) == k:
                    min_util = topk_queue.sort()[k - 1][0]

        i = secondary.index(pri_item)
        if i != -1 and i + 1 < len(secondary):
            second_item = secondary[i + 1]
            if util < min_util and second_item.utility < 0:
                continue
            beta_dp = find_database_projection(beta, project_db)
            new_primary = list()
            new_secondary = list()
            for item in secondary:
                if item != pri_item and item.twu >= min_util:
                    p, twu = calculate_probability_and_twu_of_item_set({item}, beta_dp)
                    lu = calculate_local_utility(beta, item, beta_dp)
                    if round(p, 3) >= user_prob_threshold and lu >= min_util:
                        new_secondary.append(item)
                        su = calculate_subtree_utility(beta, item, beta_dp)
                        if su >= min_util:
                            new_primary.append(item)
            efim_global_search(
                beta,
                beta_dp,
                new_primary,
                new_secondary,
                min_util,
                user_prob_threshold,
                k,
                topk_queue,
            )

In [ ]:
def create_utility_bin_array(item_set: list[Item], database: list[Transaction]):
    n = len(item_list)
    twu_array = [0 for i in range(n)]
    for i in range(n):
        twu_array[i] = calculate_transaction_weight_utility({i}, database)
    return twu_array

In [ ]:
def topk_mining_based_on_EFIM(
    database: list[Transaction], item_list: set[Item], k: int, min_prob: float
):
    if k <= 0:
        return
    
    alpha: set[Item] = set()
    topk_queue = PriorityQueue(k)
    min_util = 0
    user_prob_threshold = min_prob * len(database)

    for item in item_list:
        item.twu = calculate_transaction_weight_utility({item}, database)
    
    positive_list: list[Item] = [item for item in item_list if item.utility >= 0]
    min_util: int = priu_pruning(positive_list, k, database)  # Algorithm 1

    secondary = list()
    for i in item_list:
        p, twu = calculate_probability_and_twu_of_item_set({i}, database)
        if round(p, 3) >= user_prob_threshold and twu >= min_util:
            secondary.append(i)

    removed = set(item_list).difference(secondary)
    
    for trans in database:
        trans.trans_items = [
            item for item in trans.trans_items if item.item not in removed
        ]

    lius: dict[frozenset[Item], int] = create_liu_dict(positive_list, database)
    
    min_util = pliue_strategy(lius, k, min_util)

    primary: list[Item] = [
        item
        for item in secondary
        if calculate_subtree_utility(alpha, item, database) >= min_util
    ]

    secondary = sort_items_by_twu_and_utility(secondary)

    efim_global_search(
        alpha,
        database,
        primary,
        secondary,
        min_util,
        user_prob_threshold,
        k,
        topk_queue,
    )

    return topk_queue

**BASE ON PHUI**

In [ ]:
def find_tuple_by_trans_id(P: AbstractList, target_trans_id: int) -> Utilities:
    utilities_list: list[Utilities] = P.utility_values
    for iTuple in utilities_list:
        if iTuple.tid == target_trans_id:
            return iTuple
    return None

def utility_list_construct(P: AbstractList, Px: AbstractList, Py: AbstractList):
    x = Px.items
    y = Py.items
    xy = x | y
    utilities_list: list[Utilities] = list()
    Pxy = AbstractList(xy, utilities_list)
    utilities_list_of_px: list[Utilities] = Px.utility_values
    for xTuple in utilities_list_of_px:
        yTuple: Utilities = find_tuple_by_trans_id(Py, xTuple.tid)
        if yTuple is not None:
            if P.utility_values:
                pTuple: Utilities = find_tuple_by_trans_id(P, xTuple.tid)
                pro = 0.00001 if pTuple.pro == 0 else pTuple.pro
                xyTuple: Utilities = Utilities(
                    xTuple.tid,
                    xTuple.pro * yTuple.pro / pro,
                    xTuple.pu + yTuple.pu - pTuple.pu,
                    xTuple.nu + yTuple.nu - pTuple.nu,
                    yTuple.ru,
                )
                utilities_list.append(xyTuple)
            else:
                xyTuple: Utilities = Utilities(
                    xTuple.tid,
                    xTuple.pro * yTuple.pro,
                    xTuple.pu + yTuple.pu,
                    xTuple.nu + yTuple.nu,
                    yTuple.ru,
                )
                utilities_list.append(xyTuple)
    return Pxy

def covl_construct(sorted_list: list[Item], eucst_dict: dict[frozenset[Item], int], database: list[Transaction]):
    covl_list = list()
    for i in range(len(sorted_list)):
        x: Item = sorted_list[i]
        coverage_list = list()
        for j in range(i + 1, len(sorted_list)):
            y: Item = sorted_list[j]
            xy = {x, y}
            key = frozenset(xy)
            xy_twu = eucst_dict.get(key)
            if x.twu == xy_twu:
                coverage_list.append(y)
        r = len(coverage_list)
        if r == 0:
            coverage_list.append(-1)
        else:
            util: int = 0
            for z in range(0, r):
                util += calculate_utility_of_item_set_in_database(
                    {x, coverage_list[z]}, database
                )
            util -= (r - 1) * calculate_utility_of_item_set_in_database({x}, database)
            covl_list.append(util)
    covl_list.sort(reverse=True)
    
    return covl_list

In [ ]:
def create_eucst_dict(
    sorted_item_list: list[Item], database: list[Transaction], min_util: int
) -> dict[frozenset[Item], int]:
    n = len(sorted_item_list)
    eucs_dict = {}
    for i in range(n):
        for j in range(n):
            if j > i:
                item_pair = frozenset({sorted_item_list[i], sorted_item_list[j]})
                twu_value = calculate_transaction_weight_utility(item_pair, database)
                if twu_value >= min_util:
                    eucs_dict[item_pair] = twu_value
    return eucs_dict

def update_eucst_dict(
    eucst_dict: dict[frozenset, int], min_util: int
) -> dict[frozenset, int]:
    return {key: twu for key, twu in eucst_dict.items() if twu >= min_util}

def create_cudm_dict(
    sorted_item_list: list[Item], database: list[Transaction]
) -> dict[frozenset[Item], int]:
    n = len(sorted_item_list)
    eucs_dict = {}
    for i in range(n):
        for j in range(n):
            if j > i:
                item_pair = frozenset({sorted_item_list[i], sorted_item_list[j]})
                util_value = calculate_utility_of_item_set_in_database(
                    {sorted_item_list[i], sorted_item_list[j]}, database
                )
                eucs_dict[item_pair] = util_value
    return eucs_dict

In [ ]:
def phui_searching_procedure(
    PList: AbstractList,
    lists: list[AbstractList],
    current_min_util: int,
    user_prob_threshold: float,
    database: list[Transaction],
    eucs_dict: dict[frozenset[Item], int],
    k: int,
    topk_queue: PriorityQueue,
):
    for i in range(0, len(lists)):
        XList: AbstractList = lists[i]
        XList_utility = XList.get_pu() + XList.get_nu()
        if (
            round(XList.get_pro(), 3) >= user_prob_threshold
            and XList_utility >= current_min_util
        ):
            if frozenset(XList.items) not in {
                frozenset(item[1]) for item in topk_queue.heap
            }:
                topk_queue.push(XList_utility, XList.items)
                if len(topk_queue.heap) == k:
                    current_min_util = topk_queue.sort()[k - 1][0]

        if (
            round(XList.get_pro(), 3) >= user_prob_threshold
            and XList.get_pu() + XList.get_ru() >= current_min_util
        ):
            new_lists: list[AbstractList] = list()
            for j in range(i + 1, len(lists)):
                YList: AbstractList = lists[j]
                x = XList.items.difference(PList.items)
                y = YList.items.difference(PList.items)
                key = frozenset(x | y)
                twu_value = eucs_dict.get(key)
                if twu_value != None and twu_value >= current_min_util:
                    ZList = utility_list_construct(PList, XList, YList)
                    if round(ZList.get_pro(), 3) >= user_prob_threshold:
                        new_lists.append(ZList)
            phui_searching_procedure(
                XList,
                new_lists,
                current_min_util,
                user_prob_threshold,
                database,
                eucs_dict,
                k,
                topk_queue,
            )

In [ ]:
def topk_mining_based_on_PHUI(
    database: list[Transaction], item_list: list[Item], k: int, min_prob: float
):
    # create priority to contains top-k HUI
    topk_queue = PriorityQueue(k)
    min_util = 0
    user_prob_threshold = min_prob * len(database)
    # first update min_util = the k-th highest utility value (using RIU strategy)
    positive_list: list[Item] = [item for item in item_list if item.utility >= 0]
    current_min_util: int = priu_pruning(  # Algorithm 1
        positive_list, min_util, database
    )
    # create a list that contains all items is unqualified
    removed_list: list[Item] = list()
    for item in item_list:
        prob, twu = calculate_probability_and_twu_of_item_set({item}, database)
        if round(prob, 3) >= user_prob_threshold and twu >= current_min_util:
            item.twu = twu
        else:
            removed_list.append(item)
    # Remove unqualified item
    new_distinct_items = [item for item in item_list if item not in removed_list]
    # sort item list by order
    new_distinct_items = sort_items_by_twu_and_utility(new_distinct_items)
    # remove unqualified items from transaction
    for trans in database:
        trans.trans_items = [
            item for item in trans.trans_items if item.item not in removed_list
        ]
    # create list[AbstractList],
    lists: list[AbstractList] = list()
    # create pnu-list
    for item in new_distinct_items:
        utility_values_list: list[tuple] = list()
        pnu_list = AbstractList({item}, utility_values_list)
        for trans in database:
            if trans.contains_item_set({item}):
                pro = trans.get_probability_of_item_set({item})
                pu = trans.get_positive_utility_of_item_set({item})
                nu = trans.get_negative_utility_of_item_set({item})
                ru = trans.get_remaining_utility_of_item_set({item})
                utility_values: Utilities = Utilities(trans.id, pro, pu, nu, ru)
                utility_values_list.append(utility_values)
        lists.append(pnu_list)

    # Create EUCST dict that contain twu > current_min_util of all 2-item-set
    eucst_dict: dict[frozenset[Item], int] = create_eucst_dict(
        new_distinct_items, database, current_min_util
    )
    # Create CUDM dict that contain utility of all 2-item-set
    cudm_dict: dict[frozenset[Item], int] = create_cudm_dict(
        new_distinct_items, database
    )
    if cudm_dict:
        cud: int = pliue_strategy(
            cudm_dict, k, current_min_util
        )  # get the k-th highest utility in CUDM dict
        current_min_util = max(current_min_util, cud)

    covl: list[int] = covl_construct(new_distinct_items, eucst_dict, database)
    if covl:
        covl_length = len(covl)
        if k > covl_length:
            current_min_util = max(current_min_util, covl[covl_length - 1])
        else:
            current_min_util = max(current_min_util, covl[k - 1])

    eucst_dict = update_eucst_dict(eucst_dict, current_min_util)
    root = AbstractList({}, list())
    phui_searching_procedure(
        root,
        lists,
        current_min_util,
        user_prob_threshold,
        database,
        eucst_dict,
        k,
        topk_queue,
    )
    return topk_queue

**Tree Base**

In [ ]:
class PHUNode:
    def __init__(
        self,
        item_set: set[Item] = None,
        utility: int = 0,
        ru: int = 0,
        prob: float = 0.0,
        children: list["PHUNode"] = None,
        parent: "PHUNode" = None,
    ):
        self.item_set = item_set if item_set is not None else set()
        self.utility = utility
        self.ru = ru
        self.prob = prob
        self.children = children if children is not None else []
        self.parent = parent

    def add_child(self, child: "PHUNode") -> None:
        self.children.append(child)

    def is_leaf(self) -> bool:
        return len(self.children) == 0

    def get_total_utility(self) -> int:
        total = self.utility
        for child in self.children:
            total += child.get_total_utility()
        return total

    def get_right_sibling(self):
        if self.parent is None:
            return []

        siblings = self.parent.children

        if not siblings or self not in siblings:
            return []

        current_index = siblings.index(self)
        if current_index < len(siblings) - 1:
            return siblings[current_index + 1 :]
        else:
            return []

    def is_htwui(
        self, database: list[Transaction], min_util: int, user_prob_threshold: float
    ) -> bool:
        p, twu = calculate_probability_and_twu_of_item_set(self.item_set, database)
        return twu >= min_util and round(p, 3) >= user_prob_threshold

    def __repr__(self) -> str:
        return f"{self.item_set}"

    def print_children(self, level=0):
        print("  " * level + f"{self.item_set}")
        for child in self.children:
            child.print_children(level + 1)

In [ ]:
def build_subtree(
    node_x: PHUNode,
    database: list[Transaction],
    current_min_util: int,
    user_prob_threshold: float,
    k: int,
    topk_queue: PriorityQueue,
):
    if not node_x.is_htwui(database, current_min_util, user_prob_threshold):
        return
    right_siblings = node_x.get_right_sibling()
    generates: list[PHUNode] = list()
    for node_y in right_siblings:
        if node_y.is_htwui(database, current_min_util, user_prob_threshold):
            xy = (node_x.item_set - node_y.item_set).union(
                node_y.item_set - node_x.item_set
            )
            xy_twu = calculate_transaction_weight_utility(xy, database)
            # EUCP Pruning
            if xy_twu >= current_min_util:
                z_item_set = node_x.item_set.union(node_y.item_set)
                z_p, z_u, z_ru, twu = calculate_utilities_of_item_set(
                    z_item_set, database
                )
                node_z = PHUNode(z_item_set, z_u, z_ru, z_p, [], node_x)
                node_x.add_child(node_z)
                if round(z_p, 3) >= user_prob_threshold:
                    generates.append(node_z)
                    if z_u >= current_min_util:
                        if frozenset(z_item_set) not in {
                            frozenset(item[1]) for item in topk_queue.heap
                        }:
                            topk_queue.push(z_u, z_item_set)
                            if len(topk_queue.heap) == k:
                                current_min_util = topk_queue.sort()[k - 1][0]
    for gen_node in generates:
        build_subtree(
            gen_node, database, current_min_util, user_prob_threshold, k, topk_queue
        )

In [ ]:
def build_tree(
    database: list[Transaction], item_list: list[Item], k: int, min_prob: float
):
    min_util = 0
    user_prob_threshold = min_prob * len(database)
    topk_queue: PriorityQueue = PriorityQueue(k)

    positive_list: list[Item] = [item for item in item_list if item.utility >= 0]
    current_min_util: int = priu_pruning(positive_list, min_util, database)
    removed_list = set()
    for item in item_list:
        prob, twu = calculate_probability_and_twu_of_item_set({item}, database)
        if round(prob, 3) >= user_prob_threshold and item.twu >= current_min_util:
            item.twu = twu
        else:
            removed_list.add(item)
    # Remove unqualified item
    new_distinct_items = [item for item in item_list if item not in removed_list]
    # sort item list by order
    new_distinct_items = sort_items_by_twu_and_utility(new_distinct_items)
    root: PHUNode = PHUNode()
    for item in new_distinct_items:
        new_node = PHUNode({item}, 0, 0, 0, list(), root)
        root.add_child(new_node)
        utility = calculate_utility_of_item_set_in_database({item}, database)
        if utility >= current_min_util:
            if frozenset({item}) not in {
                frozenset(item[1]) for item in topk_queue.heap
            }:
                topk_queue.push(utility, {item})
                if len(topk_queue.heap) == k:
                    current_min_util = topk_queue.sort()[k - 1][0]

    liu_dict: dict[frozenset[Item], int] = create_liu_dict(new_distinct_items, database)
    pliue_util = pliue_strategy(liu_dict, k, min_util)
    current_min_util = max(current_min_util, pliue_util)
    for child in root.children:
        build_subtree(
            child, database, current_min_util, user_prob_threshold, k, topk_queue
        )
    return topk_queue

In [ ]:
item_list = [Item(f"i{i+1}", random.randint(-100, 100)) for i in range(200)]

db1 = list()
db2 = list()
db3 = list()
for transaction_id in range(1, 1001):
    trans_items = set()
    selected_items = random.sample(item_list, random.randint(20, 40))
    for item in selected_items:
        quantity = random.randint(1, 100)
        probability = round(random.uniform(0.1, 1.0), 2)
        trans_item = TransItem(item, quantity, probability)
        trans_items.add(trans_item)

    transaction = Transaction(transaction_id, trans_items)
    db1.append(transaction)
    db2.append(transaction)
    db3.append(transaction)

In [ ]:
# t0 = datetime.datetime.now()
# rs = topk_mining_based_on_EFIM(db1, item_list, 20, 0.1)
# rs.print_items()
# t1 = datetime.datetime.now()
# print("topk_mining_based_on_EFIM: " + str(len(rs.heap)))
# print("topk_mining_based_on_EFIM: " + str(t1 - t0))

# t1 = datetime.datetime.now()
# rs2 = topk_mining_based_on_PHUI(db2, item_list, 20, 0.01)
# rs2.print_items()
# t2 = datetime.datetime.now()
# print("topk_mining_based_on_PHUI: " + str(len(rs2.heap)))
# print("topk_mining_based_on_PHUI: " + str(t2 - t1))
# rs3 = build_tree(db3, item_list, 100, 0.05)
# print("done")